<a href="https://colab.research.google.com/github/Akshatgarg221/fac/blob/main/copy_of_yet_another_copy_of_untitled22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def plot_line_graph(df, column_name):

    """
    Plots a line graph for a given column in a DataFrame using Plotly.
    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    column_name (str): The column name to plot.
    """

    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' does not exist in the DataFrame.")

    # Create a line plot using Plotly Express
    fig = px.line(df, y=column_name, title=f'Line Graph for {column_name}')

    # Show the plot
    fig.show()

In [ ]:
from google.colab import drive
import pandas as pd
import yfinance as yf

# Mount Google Drive
drive.mount('/content/drive')

# Path to your CSV file
file_path = '/content/drive/My Drive/data_2020_2023_1d.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)


data=yf.download('^IXIC','2020-01-01','2024-01-02')

[*********************100%%**********************]  1 of 1 completed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data.reset_index(inplace=True)

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

# Extract the date part
df['date'] = df['datetime'].dt.date
# data['date']=pd.to_datetime(data.index)
data['date']=data['Date'].dt.date

In [ ]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,date
0,2020-01-02,9039.459961,9093.429688,9010.889648,9092.190430,9092.190430,2862700000,2020-01-02
1,2020-01-03,8976.429688,9065.759766,8976.429688,9020.769531,9020.769531,2586520000,2020-01-03
2,2020-01-06,8943.500000,9072.410156,8943.500000,9071.469727,9071.469727,2810450000,2020-01-06
3,2020-01-07,9076.639648,9091.929688,9042.549805,9068.580078,9068.580078,2381740000,2020-01-07
4,2020-01-08,9068.030273,9168.889648,9059.379883,9129.240234,9129.240234,2472620000,2020-01-08


In [ ]:

j=0
df['marketopen']=0
for i in range(len(df)):
  if(j<1006):
    if(df['date'][i]==data['date'][j]):
      df['marketopen'][i]=1
      j+=1

In [ ]:
df.head()

,datetime,open,high,low,close,volume,date,marketopen
0,2020-01-01 05:30:00,7195.24,7255.0,7175.15,7200.85,16792.388165,2020-01-01,0
1,2020-01-02 05:30:00,7200.77,7212.5,6924.74,6965.71,31951.483932,2020-01-02,1
2,2020-01-03 05:30:00,6965.49,7405.0,6871.04,7344.96,68428.500451,2020-01-03,1
3,2020-01-04 05:30:00,7345.00,7404.0,7272.21,7354.11,29987.974977,2020-01-04,0
4,2020-01-05 05:30:00,7354.19,7495.0,7318.00,7358.75,38331.085604,2020-01-05,0


In [ ]:
def Heiken_Ashi(df):
    df['HA_Close']=(df['open']+ df['high']+ df['low']+ df['close'])/4
    df['HA_Open']=(df['open']+df['close'])/2

    for i in range(1, len(df)):
        df['HA_Open'][i]=(df['HA_Open'][i-1]+df['HA_Close'][i-1])/2
    df['HA_High']=df[['HA_Open','HA_Close','high']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','low']].min(axis=1)
    return

Heiken_Ashi(df)

In [ ]:
def calculate_macd(df,x, short_window=12, long_window=26, signal_window=9):
    """
    Calculate MACD, Signal line, and MACD Histogram.

    Parameters:
    df (pd.DataFrame): DataFrame containing the stock data with 'Close' price.
    short_window (int): The short window period for EMA (default is 12).
    long_window (int): The long window period for EMA (default is 26).
    signal_window (int): The window period for the Signal line EMA (default is 9).

    Returns:
    df (pd.DataFrame): DataFrame with MACD, Signal line, and Histogram.
    """
    # Calculate the Short and Long EMA
    df['EMA_short'] = df[x].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df[x].ewm(span=long_window, adjust=False).mean()

    # Calculate the MACD line
    # df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['MACD'] =df['HA_Low'].ewm(span = short_window,adjust=False).mean() + df['HA_Close'].ewm(span = short_window,adjust=False).mean()+ df['HA_Open'].ewm(span = short_window,adjust=False).mean()+ df['HA_High'].ewm(span = short_window,adjust=False).mean()- df['HA_Close'].ewm(span=long_window,adjust=False).mean()-df['HA_Open'].ewm(span=long_window ,adjust=False).mean()-df['HA_High'].ewm(span = long_window ,adjust=False).mean()-df['HA_Low'].ewm(span = long_window,adjust=False).mean()

    # Calculate the Signal line
    df['Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()

    # Calculate the MACD Histogram
    df['Hist'] = df['MACD'] - df['Signal']

    return df

# def generate_macd_signals(df):
#     df['Signal1'] = 0
#     for i in range(len(df)):
#         if df['MACD'][i] > df['Signal'][i] and df['MACD'][i-1]<=df['Signal'][i-1]:
#             df['Signal1'][i] = 1
#         elif df['MACD'][i] < df['Signal'][i] and df['MACD'][i-1]>=df['Signal'][i-1]:
#             df['Signal1'][i] = -1
#         else:
#             df['Signal1'][i] = 0
#     return df

def generate_macd_signals(df):
    df['Signal1'] = 0
    for i in range(len(df)):
        if df['MACD'][i] > df['Signal'][i] :
            df['Signal1'][i] = 1
        elif df['MACD'][i] < df['Signal'][i]:
            df['Signal1'][i] = -1
        else:
            df['Signal1'][i] = 0
    return df
# df = calculate_macd(df)
# df = generate_macd_signals(df)
df.head()



,datetime,open,high,low,close,volume,date,marketopen,HA_Close,HA_Open,HA_High,HA_Low
0,2020-01-01 05:30:00,7195.24,7255.0,7175.15,7200.85,16792.388165,2020-01-01,0,7206.5600,7198.045000,7255.0,7175.150000
1,2020-01-02 05:30:00,7200.77,7212.5,6924.74,6965.71,31951.483932,2020-01-02,1,7075.9300,7202.302500,7212.5,6924.740000
2,2020-01-03 05:30:00,6965.49,7405.0,6871.04,7344.96,68428.500451,2020-01-03,1,7146.6225,7139.116250,7405.0,6871.040000
3,2020-01-04 05:30:00,7345.00,7404.0,7272.21,7354.11,29987.974977,2020-01-04,0,7343.8300,7142.869375,7404.0,7142.869375
4,2020-01-05 05:30:00,7354.19,7495.0,7318.00,7358.75,38331.085604,2020-01-05,0,7381.4850,7243.349688,7495.0,7243.349688


In [ ]:
def calculateibs(df, period):
    df['IBS'] = (df['close']-df['low'])/(df['high']-df['low'])

    df['weight_avg'] = df['IBS'].ewm(span=period, adjust=False).mean()
    df['sigi']=0
    for i in range(len(df)):
      if(df['weight_avg'][i] > 0.7) :
        df['sigi'][i] = 1
      elif(df['weight_avg'][i] < 0.3):
        df['sigi'][i] = -1


    return df
df=calculateibs(df,5)
# print(len(df[df['weight_avg']>0.975]))
# print(df['weight_avg'].min())
# print(df['weight_avg'].mean())

In [ ]:
def bolingerband(df,x,window=20,std=2):
   df['MA'] = df[x].rolling(window=window).mean()
   df['STD'] = df[x].rolling(window=window).std()
   df['Upper Band'] = df['MA'] + (df['STD'] * std)
   df['Lower Band'] = df['MA'] - (df['STD'] * std)

   df['Signal2']=0
   for i in range(len(df)):
    if(df['denoised2'][i]>df['Upper Band'][i]):
      df['Signal2'][i]=-1
    elif(df['denoised2'][i]<df['Lower Band'][i]):
      df['Signal2'][i]=1
    else:
      df['Signal2'][i]=0
   return df
df.dropna(inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
!pip install filterpy
from filterpy.kalman import KalmanFilter
def denoise(df):#using pca filter
  cf=pd.DataFrame(df['close'])
  cf['open']=df['open']
  cf['high']=df['high']
  cf['low']=df['low']
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(cf)
  pca = PCA(n_components=4)
  pca.fit(scaled_data)
  principalComponent = pca.transform(scaled_data)
  principalDf = pd.DataFrame(data=principalComponent, columns=['imp', 'principal component 2','a','b'])
  return principalDf['imp']

def denoise2(df,para):#using kalman filter
  prices = df[para].values


  kf = KalmanFilter(dim_x=2, dim_z=1)
  kf.x = np.array([prices[0], 0])
  kf.F = np.array([[1, 1], [0, 1]])
  kf.H = np.array([[1, 0]])
  kf.P *= 1000
  kf.R = 1
  kf.Q = np.array([[0.001, 0], [0, 0.001]])

  newprices = []
  for price in prices:
      kf.predict()
      kf.update(price)
      newprices.append(kf.x[0])
  return newprices

df['denoised']=denoise(df)
df['denoised2']=denoise2(df,'close')
df['denoised3']=denoise2(df,'denoised')
df=calculate_macd(df,'denoised2',12,26,9)
# df = calculate_macd(df)
df = generate_macd_signals(df)
df=bolingerband(df,'denoised2')
df.head()




,datetime,open,high,low,close,volume,date,marketopen,HA_Close,HA_Open,...,sigr,Sig1,Sig2,TR,ATR,rolling,rclose,signals,pvalue,stragegy
0,2020-01-01 05:30:00,7195.24,7255.0,7175.15,7200.85,16792.388165,2020-01-01,0,7206.5600,7198.045000,...,0,0,0,79.85,79.850000,7200.850000,7200.850000,0,100000.0,2
1,2020-01-02 05:30:00,7200.77,7212.5,6924.74,6965.71,31951.483932,2020-01-02,1,7075.9300,7202.302500,...,1,-1,0,287.76,183.805000,7083.280000,7083.280000,0,100000.0,2
2,2020-01-03 05:30:00,6965.49,7405.0,6871.04,7344.96,68428.500451,2020-01-03,1,7146.6225,7139.116250,...,1,0,0,533.96,300.523333,7170.506667,7170.506667,0,100000.0,2
3,2020-01-04 05:30:00,7345.00,7404.0,7272.21,7354.11,29987.974977,2020-01-04,0,7343.8300,7142.869375,...,-1,0,0,131.79,258.340000,7216.407500,7221.593333,0,100000.0,2
4,2020-01-05 05:30:00,7354.19,7495.0,7318.00,7358.75,38331.085604,2020-01-05,0,7381.4850,7243.349688,...,-1,0,0,177.00,242.072000,7244.876000,7352.606667,0,100000.0,2


In [ ]:
y='close'
df['200_ema']= df[y].ewm(span=200, adjust=False).mean()
df['100_ema']=df[y].ewm(span=100, adjust=False).mean()
df['100m']=df['100_ema']*1.05
df['100m2']=df['100_ema']*0.85
df['sig200']=0
for i in range(len(df)):
  if(df['200_ema'][i]>df[y][i]):
    df['sig200'][i]=-1
  elif(df['200_ema'][i]<df[y][i]):
    df['sig200'][i]=1
  else:
    df['sig200'][i]=0


fig = go.Figure()

# Add the 'close' line
fig.add_trace(go.Scatter(x=df.index, y=df[y], mode='lines', name='Close Price'))

# Add the '200_ema' line
# fig.add_trace(go.Scatter(x=df.index, y=df['200_ema'], mode='lines', name='200 EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df['100_ema'], mode='lines', name='2100 EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df['100m'], mode='lines', name='2100 EMA'))
fig.add_trace(go.Scatter(x=df.index, y=df['100m2'], mode='lines', name='2100dfs EMA'))

# Update layout
fig.update_layout(
    title='Close Price and 200 EMA',
    xaxis_title='Date',
    yaxis_title='Price',
    template='plotly_dark'
)
# Add upward triangles for sig1 == 1
# fig.add_trace(go.Scatter(
#     x=df[df['Sig2'] == 1].index,
#     y=df[df['Sig2'] == 1][y],
#     mode='markers',
#     marker=dict(symbol='triangle-up', size=10, color='green'),
#     name='Buy Signal'
# ))

# # Add downward arrows for sig1 == -1
# fig.add_trace(go.Scatter(
#     x=df[df['Sig2'] == -1].index,
#     y=df[df['Sig2'] == -1][y],
#     mode='markers',
#     marker=dict(symbol='arrow-down', size=10, color='red'),
#     name='Sell Signal'
# ))


# Show the plot
fig.show()

In [ ]:
def calculatersi(df, period):
    df['pricechange'] =df['denoised'].diff()
    df['gain'] = df['pricechange'].apply(lambda x: x if x > 0 else 0)
    df['loss'] = df['pricechange'].apply(lambda x: abs(x) if x < 0 else 0)

    df['avggain'] = df['gain'].rolling(window=period, min_periods=1).mean()
    df['avgloss'] = df['loss'].rolling(window=period, min_periods=1).mean()

    df['RS'] = df['avggain']/df['avgloss']
    df['RSI'] = 100-(100/(1+df['RS']))

    df.drop(['pricechange', 'gain', 'loss', 'avggain', 'avgloss', 'RS'], axis=1, inplace=True)

    df['sigr'] = 0
    for i in range(len(df)):
      if(df['RSI'][i] < 50) :
        df['sigr'][i] = 1
      elif(df['RSI'][i] > 50) :
        df['sigr'][i] = -1

    return df
df=calculatersi(df,14)

In [ ]:
df['Sig1']=0
df['Sig2']=0
for i in range(len(df)):#combining both macd and marabozu
  if(df['Signal1'][i]==1 and df['sigi'][i]==1  ):
    df['Sig1'][i]=1
  if(df['Signal1'][i]==-1 and df['sigi'][i]==-1 ):
    df['Sig1'][i]=-1

for i in range(len(df)):#combining both macd and marabozu
  if(df['Signal2'][i]==1 and df['sigr'][i]==1 ):
    df['Sig2'][i]=1
  if(df['Signal2'][i]==-1 and df['sigr'][i]==-1):
    df['Sig2'][i]=-1


In [ ]:
len(df[df['Sig1']>0])

86

In [ ]:
def atr(data,period=14):
    data['High-Low'] = data['high'] - data['low']
    data['High-PrevClose'] = abs(data['high'] - data['close'].shift(1))
    data['Low-PrevClose'] = abs(data['low'] - data['close'].shift(1))

    # True Range is the maximum of the three calculated ranges
    data['TR'] = data[['High-Low', 'High-PrevClose', 'Low-PrevClose']].max(axis=1)

    # Calculate ATR
    data['ATR'] = data['TR'].rolling(window=period, min_periods=1).mean()

    # Drop the intermediate columns used for calculation
    data = data.drop(['High-Low', 'High-PrevClose', 'Low-PrevClose'], axis=1)

    return data


In [ ]:
def alternate(start,end,stragegy):#checking results of other stragegy
  amount = 100000
  position=0
  no=0
  trade=0
  enterprice=0
  for i in range(start,end+1):
    price=df['close'][i]

    if(stragegy==1 ):#for stragegy one defining singal 1 and 2
      sig1=df['Sig1'][i]
      sig2=df['Sig2'][i]

    if(stragegy==2 ):#for stragegy two defining singal 1 and 2
      sig1=df['Sig2'][i]
      sig2=df['Sig1'][i]

    if(position==0 and sig1==1 ):
      no=amount/price
      amount=-amount*0.0015
      enterprice=price
      position=1
      trade+=1

    if(position==1 and sig2==-1):
      amount=no*price
      position=0
      trade+=1

    if(position==0 and sig1==-1):
      no=amount/price
      enterprice=price
      amount-=amount*0.0015
      position=-1
      trade+=1

    if(position==-1 and sig2==1):
      amount+=no*enterprice-no*price
      position=0
      trade+=1

    if(position==1 and price<enterprice*(1-stoploss_longtrademo/100) and df['marketopen'][i]==1):
      amount=no*price
      position=0
      trade+=1

    if(position==1 and price<enterprice*(1-stoploss_longtrademc/100) and df['marketopen'][i]==0):
      amount=no*price
      position=0
      trade+=1

    if(position==-1 and price>enterprice*(1+stoploss_shorttrademo/100) and df['marketopen'][i]==1):
      amount+=no*enterprice-no*price
      position=0
      trade+=1

    if(position==-1 and price>enterprice*(1+stoploss_shorttrademc/100) and df['marketopen'][i]==0):
      amount+=no*enterprice-no*price
      position=0
      trade+=1



    # if(position==1 and price>enterprice*(1+takeprofit_longtrade/100)):
    #   amount=no*price
    #   position=0
    #   trade+=1

    # if(position==-1 and price<enterprice*(1-takeprofit_shorttrade/100)):
    #   amount+=no*enterprice-no*price
    #   position=0
    #   trade+=1


    # if(position==1 and price<df['100m2'][i]):
    #   amount=no*price
    #   position=0
    # #   trade+=1

    if(position==1 and df['rclose'][i]<df['rolling'][i]-atrmo*df['ATR'][i] and df['marketopen'][i]==1):
      amount=no*price
      position=0
      trade+=1

    if(position==1 and df['rclose'][i]<df['rolling'][i]-atrmc*df['ATR'][i] and df['marketopen'][i]==0):
      amount=no*price
      position=0
      trade+=1

    if(position==-1 and df['rclose'][i]>df['rolling'][i]+atrsmo*df['ATR'][i] and df['marketopen'][i]==1):
      amount+=no*enterprice-no*price
      position=0
      trade+=1

    if(position==-1 and df['rclose'][i]>df['rolling'][i]+atrsmc*df['ATR'][i] and df['marketopen'][i]==0):
      amount+=no*enterprice-no*price
      position=0
      trade+=1

    # if(position==-1 and df['rclose'][i]<df['rolling'][i]+atrm*df['ATR'][i]):
    #   amount=no*enterprice-no*price
    #   position=0
    #   trade+=1





  if(position==1):
    amount+=no*price
  if(position==-1):
    amount+=no*enterprice-no*price

  return (amount-100000)/100000

In [ ]:
def backtester(df):
  from itertools import starmap

  amounta=[]
  x=0
  df['signals']=0
  amount = 100000
  position=0
  df['pvalue']=100000
  df['stragegy']=0
  drawdown=[]
  maxp=0
  no=0
  enteryindex=[]
  exitindex=[]
  trade=0
  stragegy=2
  enterprice=0
  enterindex=0
  for i in range(len(df)):
    price=df['close'][i]
    df['stragegy'][i]=stragegy

    if(stragegy==1 ):#for stragegy one defining singal 1 and 2
      sig1=df['Sig1'][i]
      sig2=df['Sig2'][i]

    if(stragegy==2 ):#for stragegy two defining singal 1 and 2
      sig1=df['Sig2'][i]
      sig2=df['Sig1'][i]


    if(position==1):
      amounta.append(no*price)
      df['pvalue'][i]=amount+no*price
    if(position==-1):
      amounta.append(amount+no*enterprice-no*price)
      df['pvalue'][i]=amount+no*enterprice-no*price
    if(position==0):
      amounta.append(amount)
      if(i):
        df['pvalue'][i]=df['pvalue'][i-1]


    if(i):
      if(df['pvalue'][i]>maxp):
        maxp=df['pvalue'][i]

    if(len(drawdown)!=0):
      x+=1
      if(drawdown[-1]<( maxp-df['pvalue'][i-1])/df['pvalue'][i-1]):
        drawdown[-1]=(maxp-df['pvalue'][i-1])/df['pvalue'][i-1]


    if(position==0 and sig1==1 and sig2!=-1):
      no=amount/price
      enterprice=price
      amount=-amount*0.0015
      position=1
      trade+=1
      enterindex=i
      maxp=amount+no*price
      drawdown.append(0)
      enteryindex.append(i)
      df['signals'][i]=1



    if(position==1 and sig2==-1):
      amount=no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=-1
      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
          stragegy=1


    if(position==0 and sig1==-1 and sig2!=1):
      no=amount/price
      amount-=amount*0.0015
      enterprice=price
      position=-1
      trade+=1
      maxp=amount
      drawdown.append(0)
      enteryindex.append(i)
      df['signals'][i]=-1


    if(position==-1 and sig2==1):
      amount+=no*enterprice-no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=1
      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(enterprice-price)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
          stragegy=1


  # stopploss long trade
    if(position==1 and price<enterprice*(1-stoploss_longtrademo/100) and df['marketopen'][i]==1):
      amount=no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=-1
      # print(a
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
          stragegy=2
      # else:
      #   if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
      #     stragegy=1

    if(position==1 and price<enterprice*(1-stoploss_longtrademc/100) and df['marketopen'][i]==0):
      amount=no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=-1
      # print(a
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
          stragegy=1


  # Stoploss short trade
    if(position==-1 and price>enterprice*(1+stoploss_shorttrademo/100) and df['marketopen'][i]==1):
      amount+=no*enterprice-no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=1

      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(enterprice-price)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
          print(alternate(enterindex,i,1),(enterprice-price)/enterprice)
          stragegy=1

#stoploss short trade
    if(position==-1 and price>enterprice*(1+stoploss_shorttrademc/100) and df['marketopen'][i]==0):
      amount+=no*enterprice-no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=1

      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(enterprice-price)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
          # print(alternate(enterindex,i,1),(enterprice-price)/enterprice)
          stragegy=1



  # # TAke profit long trade LOSS LONG TRADE
  #   if(position==1 and price>enterprice*(1+takeprofit_longtrade/100)):
  #     # print(price,enterprice)
  #     amount=no*price
  #     position=0
  #     trade+=1
  #     exitindex.append(i)
  #     df['signals'][i]=-1
  #     # print(a
  #     if(stragegy==1):
  #       if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
  #         stragegy=2
  #     else:
  #       if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
  #         stragegy=1


  # #take profit short trade
  #   if(position==-1 and price<enterprice*(1-takeprofit_shorttrade/100)):
  #     amount+=no*enterprice-no*price
  #     position=0
  #     trade+=1
  #     exitindex.append(i)
  #     df['signals'][i]=1

  #     print(amount,stragegy)
  #     if(stragegy==1):
  #       if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
  #         stragegy=2
  #     else:
  #       if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
  #         stragegy=1


    # if(position==1 and price<df['100m2'][i]):
    #   amount=no*price
    #   position=0
    #   trade+=1
    #   exitindex.append(i)
    #   df['signals'][i]=-1

    #   # print(amount,stragegy)
    #   if(stragegy==1):
    #     if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
    #       stragegy=2
    #   else:
    #     if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
    #       stragegy=1


    if(position==1 and df['rclose'][i]<df['rolling'][i]-atrmo*df['ATR'][i] and df['marketopen'][i]==1):
      amount=no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=-1
      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
          stragegy=1


    if(position==1 and df['rclose'][i]<df['rolling'][i]-atrmc*df['ATR'][i] and df['marketopen'][i]==0):
      amount=no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=-1
      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
          stragegy=1




    if(position==-1 and df['rclose'][i]>df['rolling'][i]+atrsmo*df['ATR'][i] and df['marketopen'][i]==1):
      amount+=no*enterprice-no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=1

      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(enterprice-price)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
          stragegy=1

    if(position==-1 and df['rclose'][i]>df['rolling'][i]+atrsmc*df['ATR'][i] and df['marketopen'][i]==0):
      amount+=no*enterprice-no*price
      position=0
      trade+=1
      exitindex.append(i)
      df['signals'][i]=1

      # print(amount,stragegy)
      if(stragegy==1):
        if(alternate(enterindex,i,2)>(enterprice-price)/enterprice):
          stragegy=2
      else:
        if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
          stragegy=1




  if(position==1):
    amount+=no*price
    exitindex.append(len(df)-1)
    df['signals'][len(df)-1]=-1
  if(position==-1):
    amount+=no*enterprice-no*price
    exitindex.append(len(df)-1)
    df['signals'][len(df)-1]=1

  # print("amount = ",amount)
  # print("returns=",(amount-100000)/1000,'%')
  drawdown=np.array(drawdown)
  # print(drawdown.max())
  # print(drawdown.min())
  # print(drawdown.mean())
  return (amount-100000)/1000


#SET STOPLOSS PERCENTAGES
stoploss_longtrademo=.5
stoploss_longtrademc=8
stoploss_shorttrademo=6
stoploss_shorttrademc=10
stoploss_shorttrade=23423
takeprofit_longtrade=5000000
takeprofit_shorttrade=5000000
r=[]
il=[]
jl=[]
atrsmo=4.0
atrsmc=2
atrmo=3.4
atrmc=4.5
df=atr(df)
df['rolling']=df['close'].rolling(window=22,min_periods=1).mean()
df['rclose']=df['close'].rolling(window=3, min_periods=1).mean()
# for i in range(0,80):
  # for j in range(0,80)
# backtester(df)
  # for j in range(0,60):
# il.append(i)
# jl.append(j)


r=np.array(r)
# print(r.max())
# print(x)
print(backtester(df))

7118.4688341931505


In [ ]:
for i in range(len(r)):
  print(r[i])

In [ ]:
plot_line_graph(df,'pvalue')

In [ ]:
plot_line_graph(df,'close')

In [ ]:
filepath='output.csv'
hf=df[['datetime','open','high','low','close','volume','signals']]
hf.to_csv(filepath)

In [ ]:
def calculate_drawdown(portfolio_values):
    """
    Calculates the drawdown and maximum drawdown from a series of portfolio values.

    Parameters:
    portfolio_values (pd.Series): Series of portfolio values over time.

    Returns:
    pd.DataFrame: DataFrame with columns ['Portfolio Value', 'Drawdown', 'Max Drawdown']
    """

    # Calculate the rolling maximum
    df['Rolling Max'] = df['pvalue'].cummax()

    # Calculate the drawdown
    df['Drawdown'] = df['pvalue'] / df['Rolling Max'] - 1

    # Calculate the maximum drawdown
    df['Max Drawdown'] = df['Drawdown'].cummin()

    return df
df=calculate_drawdown(0)
print(df['Max Drawdown'].min())

-0.28885738601077826


In [ ]:
from itertools import starmap

amounta=[]
x=0
amount = 100000
position=0
df['pvalue']=100000
drawdown=[]
maxp=0
no=0
trade=0
stragegy=1
enterprice=0
enterindex=0
for i in range(len(df)):
  price=df['close'][i]

  if(stragegy==1 ):#for stragegy one defining singal 1 and 2
    sig1=df['Sig1'][i]
    sig2=df['Sig2'][i]

  if(stragegy==2 ):#for stragegy two defining singal 1 and 2
    sig1=df['Sig2'][i]
    sig2=df['Sig1'][i]
  if(i):
    if(df['pvalue'][i-1]>maxp):
      maxp=df['pvalue'][i-1]

  if(len(drawdown)!=0):
    x+=1
    if(drawdown[-1]<( maxp-df['pvalue'][i-1])/df['pvalue'][i-1]):
      drawdown[-1]=(maxp-df['pvalue'][i-1])/df['pvalue'][i-1]


  if(position==0 and sig1==1 ):
    no=amount/price
    enterprice=price
    amount=-amount*0.0015
    position=1
    trade+=1
    enterindex=i
    maxp=amount+no*price
    drawdown.append(0)



  if(position==1 and sig2==-1):
    amount=no*price
    position=0
    trade+=1
    # print(amount,stragegy)
    if(stragegy==1):
      if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
        stragegy=2
    else:
      if(alternate(enterindex,i,1)>(price-enterprice)/enterprice):
        stragegy=1


  if(position==0 and sig1==-1):
    no=amount/price
    amount-=amount*0.0015
    enterprice=price
    position=-1
    trade+=1
    maxp=amount
    drawdown.append(0)

  if(position==-1 and sig2==1):
    amount+=no*enterprice-no*price
    position=0
    trade+=1
    # print(amount,stragegy)
    if(stragegy==1):
      if(alternate(enterindex,i,2)>(enterprice-price)/enterprice):
        stragegy=2
    else:
      if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
        stragegy=1

  if(position==1 and price<enterprice*0.98):
    amount=no*price
    position=0
    trade+=1
    # print(a
    if(stragegy==1):
      if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
        stragegy=2
    else:
      if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
        stragegy=1

  if(position==-1 and price>enterprice*1.015):
    amount+=no*enterprice-no*price
    position=0
    trade+=1
    # print(amount,stragegy)
    if(stragegy==1):
      if(alternate(enterindex,i,2)>(price-enterprice)/enterprice):
        stragegy=2
    else:
      if(alternate(enterindex,i,1)>(enterprice-price)/enterprice):
        stragegy=1

  if(position==1):
    amounta.append(no*price)
    df['pvalue'][i]=amount+no*price
  if(position==-1):
    amounta.append(amount+no*enterprice-no*price)
    df['pvalue'][i]=amount+no*enterprice-no*price
  if(position==0):
    amounta.append(amount)
    if(i):
      df['pvalue'][i]=df['pvalue'][i-1]


if(position==1):
  amount+=no*price
if(position==-1):
  amount+=no*enterprice-no*price

print("amount = ",amount)
print("returns=",(amount-100000)/1000,'%')

amount =  2886095.1137663918
returns= 2786.095113766392 %


In [ ]:
drawdown=np.array(drawdown)
print(drawdown.max())
print(drawdown.min())
print(drawdown.mean())
print(drawdown.std())

0.3365287666404799
0.0
0.03903120640229123
0.07036622409158175


In [ ]:
amounta=np.array(amounta)
print(amounta.max())
print(amounta.min())
print(amounta.mean())
print(amounta.std())

2888559.5613827943
84238.32796719109
1286349.644013213
738919.2223269213


In [ ]:
amount = 100000
position=0
no=0
trade=0
for i in range(len(df)):
  price=df['close'][i]
  sig1=df['Signal1'][i]
  sig2=df['Signal1'][i]
  if(position==0 and sig1==1 ):
    no=amount/price
    amount=0
    position=1
    trade+=1

  if(position==1 and sig2==-1):
    amount=no*price
    position=0
    trade+=1

  if(position==0 and sig1==-1):
    no=amount/price
    enterprice=price
    position=-1
    trade+=1

  if(position==-1 and sig2==1):
    amount+=no*enterprice-no*price
    position=0
    trade+=1
if(position==1):
  amount+=no*price
if(position==-1):
  amount+=no*enterprice-no*price

print("amount = ",amount)
print("returns=",(amount-100000)/1000,'%')

amount =  451138.72635094216
returns= 351.13872635094214 %


In [ ]:
amount = 100000
position=0
no=0
trade=0
for i in range(len(df)):
  price=df['close'][i]
  sig1=df['Signal2'][i]
  sig2=df['Signal2'][i]
  if(position==0 and sig1==1 ):
    no=amount/price
    amount=0
    position=1
    trade+=1

  if(position==1 and sig2==-1):
    amount=no*price
    position=0
    trade+=1

  if(position==0 and sig1==-1):
    no=amount/price
    enterprice=price
    position=-1
    trade+=1

  if(position==-1 and sig2==1):
    amount+=no*enterprice-no*price
    position=0
    trade+=1
if(position==1):
  amount+=no*price
if(position==-1):
  amount+=no*enterprice-no*price

print("amount = ",amount)
print("returns=",(amount-100000)/1000,'%')

amount =  -84304.68272384617
returns= -184.30468272384616 %
